In [18]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [19]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

In [20]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 9s 1us/step


In [21]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [22]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [23]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [24]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [25]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [26]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
  512/60000 [..............................] - ETA: 2:51:16 - loss: 2.2010 - acc: 0.2129

/usr/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.251255). Check your callbacks.
  % delta_t_median)


60000/60000 [==============================] - 128s 2ms/step - loss: 0.2671 - acc: 0.9189 - val_loss: 0.0532 - val_acc: 0.9830
Epoch 2/12
60000/60000 [==============================] - 36s 604us/step - loss: 0.0848 - acc: 0.9747 - val_loss: 0.0384 - val_acc: 0.9870
Epoch 3/12
60000/60000 [==============================] - 36s 604us/step - loss: 0.0621 - acc: 0.9815 - val_loss: 0.0319 - val_acc: 0.9894
Epoch 4/12
60000/60000 [==============================] - 36s 603us/step - loss: 0.0526 - acc: 0.9840 - val_loss: 0.0316 - val_acc: 0.9882
Epoch 5/12
60000/60000 [==============================] - 36s 597us/step - loss: 0.0458 - acc: 0.9861 - val_loss: 0.0291 - val_acc: 0.9895
Epoch 6/12
60000/60000 [==============================] - 37s 611us/step - loss: 0.0422 - acc: 0.9874 - val_loss: 0.0302 - val_acc: 0.9900
Epoch 7/12
60000/60000 [==============================] - 36s 608us/step - loss: 0.0370 - acc: 0.9889 - val_loss: 0.0275 - val_acc: 0.9908
Epoch 8/12
60000/60000 [===============

In [28]:
model.save('conv_model.h5')

In [31]:
import onnx
import onnxmltools
onnx_model = onnxmltools.convert_keras(model)
onnx.save(onnx_model, 'onnx_conv_model.onnx')